In [1]:
from ops.imports import *
from ops.process import Align
import ops.firesnake
from ops.firesnake import Snake

# runs example from example_data/ sub-directory of project
home = os.path.dirname(os.path.dirname(ops.__file__))
os.chdir(os.path.join(home, 'example_data'))

df_design = pd.read_csv('design.csv')

THRESHOLD_STD = 300
THRESHOLD_DAPI = 1200
THRESHOLD_CELL = 800
NUCLEUS_AREA = 0.25*150, 0.25*800
WILDCARDS = dict(well='A1', tile='107')

In [2]:
search = 'input/*/10X*{well}_Tile-{tile}.tif'.format(**WILDCARDS)
input_files = natsorted(glob(search))
for f in input_files:
    print(f)

# used to format output filenames
description = parse(input_files[0])
description['subdir'] = 'process_ipynb'
description.pop('cycle');

input/10X_c1-SBS-1/10X_c1-SBS-1_A1_Tile-107.tif
input/10X_c2-SBS-2/10X_c2-SBS-2_A1_Tile-107.tif
input/10X_c3-SBS-3/10X_c3-SBS-3_A1_Tile-107.tif
input/10X_c4-SBS-4/10X_c4-SBS-4_A1_Tile-107.tif
input/10X_c5-SBS-5/10X_c5-SBS-5_A1_Tile-107.tif
input/10X_c6-SBS-6/10X_c6-SBS-6_A1_Tile-107.tif
input/10X_c7-SBS-7/10X_c7-SBS-7_A1_Tile-107.tif
input/10X_c8-SBS-8/10X_c8-SBS-8_A1_Tile-107.tif
input/10X_c9-SBS-9/10X_c9-SBS-9_A1_Tile-107.tif
input/10X_c10-SBS-10/10X_c10-SBS-10_A1_Tile-107.tif
input/10X_c11-SBS-11/10X_c11-SBS-11_A1_Tile-107.tif
input/10X_c12-SBS-12/10X_c12-SBS-12_A1_Tile-107.tif


### load, align, apply Laplacian-of-Gaussian filter (log)

In [3]:
data = np.array([read(f) for f in input_files])
aligned = Snake._align(data, method='DAPI')
save(name(description, tag='aligned'), aligned)

In [ ]:
loged = Snake._transform_log(aligned, skip_index=0)
save(name(description, tag='log'), loged)

In [ ]:
maxed = Snake._max_filter(loged, 3, remove_index=0)
save(name(description, tag='maxed'), maxed)

### detect candidate reads

In [ ]:
std = Snake._compute_std(loged, remove_index=0)
save(name(description, tag='std'), std)

In [ ]:
peaks = Snake._find_peaks(std)
save(name(description, tag='peaks'), peaks)

### segment nuclei and cells

In [ ]:
nuclei = Snake._segment_nuclei(data[0], THRESHOLD_DAPI,
 area_min=NUCLEUS_AREA[0], area_max=NUCLEUS_AREA[1])

save(name(description, tag='nuclei'), nuclei)

In [ ]:
cells = Snake._segment_cells(data[0], nuclei, THRESHOLD_CELL)
save(name(description, tag='cells'), cells)

### extract base intensity, call reads, assign to cells

In [ ]:
df_bases = Snake._extract_bases(maxed, peaks, cells, 
                        THRESHOLD_STD, wildcards=WILDCARDS)
df_bases.to_csv(name(description, tag='bases', ext='csv'), index=None)

In [ ]:
df_reads = Snake._call_reads(df_bases)
filename = name(description, tag='reads', ext='csv')
df_reads.to_csv(filename, index=None)

In [ ]:
# maintain same precision as snakemake pipeline
df_reads = pd.read_csv(filename) 
df_cells = Snake._call_cells(df_reads)
df_cells.to_csv(name(description, tag='cells', ext='csv'), index=None)